<a href="https://colab.research.google.com/github/plaban1981/Langchain_usecases/blob/main/Iamexambee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU langchain openai tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 4.4 MB/s eta 0:00:00


In [57]:
!pip list

Package                       Version
----------------------------- ---------------------
absl-py                       1.4.0
aiohttp                       3.8.4
aiosignal                     1.3.1
alabaster                     0.7.13
albumentations                1.2.1
altair                        4.2.2
anyio                         3.7.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
array-record                  0.3.0
arviz                         0.15.1
astropy                       5.2.2
astunparse                    1.6.3
async-timeout                 4.0.2
attrs                         23.1.0
audioread                     3.0.0
autograd                      1.5
Babel                         2.12.1
backcall                      0.2.0
beautifulsoup4                4.11.2
bleach                        6.0.0
blis                          0.7.9
blosc2                        2.0.0
bokeh                         2.4.3
branc

In [5]:
import langchain
from langchain.prompts.base import StringPromptTemplate
from langchain.prompts import PromptTemplate,StringPromptTemplate
from langchain.agents import Tool, AgentExecutor, AgentOutputParser,LLMSingleActionAgent,initialize_agent
from langchain import OpenAI,LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.schema import AgentAction,AgentFinish
import re
from typing import List,Union
import os
from getpass import getpass
#
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
#
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.chat_models import ChatOpenAI

In [3]:
os.environ["OPENAI_API_KEY"] = getpass()


··········


1.Prescribed word count: round 300 words

2.Student word count:  specify the number of words present in the essay content specified within 3 backticks

3.Try to stay within +/- 10 words of the prescribed word limit : If the student word count is within a + or - 10 words then mention "Well Done" otherwise mention "Word Limit not adhered"

4.Title: If tile is present then mention " Title provided" otherwise mention "No Title provided"

5. Content and Structure: Provide points between -5 to 10 based on the GRADING CRITERIA for CONTENT

6. Language: Provide the points based on the GRADING CRITERIA for LANGAUAGE AND GRAMMAR.Also provide spelling errors in RED and provide the correct answer next to the incorrect words in GREEN BOLD.

## ESSAY GRADER

In [7]:
p_word_count_schema = ResponseSchema(name="Prescribed Word Count",
                                description = "This is Word count that has been prescribed as 300 Words ")
#
s_word_count_schema = ResponseSchema(name="Student Word Count",
                                  description = "This is the word count thats specifies the number of words present in the essay content specified within 3 backticks")
#
s_word_limit_schema = ResponseSchema(name="Try to stay within +/- 10 words of the prescribed word limit",
                                     description="This is the word limit that the student has adhered the word limit or exceeded the word limit")
#
title_schema = ResponseSchema(name="Title",
                              description="Title of the Essay in case provided")
#
content_schema = ResponseSchema(name="Content and Structure",
                                description="This provides points between -5 to 10 based on the GRADING CRITERIA for CONTENT")
#
language_schema = ResponseSchema(name="Language",
                                 description="This provides the points based on the GRADING CRITERIA for LANGAUAGE AND GRAMMAR.Also provide spelling errors in RED and provide the correct answer next to the incorrect words in GREEN BOLD.")

response_schemas = [p_word_count_schema,
                    s_word_count_schema,
                    s_word_limit_schema,
                    title_schema,
                    content_schema,
                    language_schema]

In [8]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

In [9]:
format_instructions

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"Prescribed Word Count": string  // This is Word count that has been prescribed as 300 Words \n\t"Student Word Count": string  // This is the word count thats specifies the number of words present in the essay content specified within 3 backticks\n\t"Try to stay within +/- 10 words of the prescribed word limit": string  // This is the word limit that the student has adhered the word limit or exceeded the word limit\n\t"Title": string  // Title of the Essay in case provided\n\t"Content and Structure": string  // This provides points between -5 to 10 based on the GRADING CRITERIA for CONTENT\n\t"Language": string  // This provides the points based on the GRADING CRITERIA for LANGAUAGE AND GRAMMAR.Also provide spelling errors in RED and provide the correct answer next to the incorrect words in GREEN BOLD.\n}\n```'

In [10]:
template ="""
Evaluate the context of the essay specified in 3 bacticks on the following criteria:
ESSAY CRITERIA:
1. TITLE : The essay must have a title.
2. WORD LIMIT: The word count in the essay must be between 290 to 310 words
3. CONTENT : The content must have the matter as per the current developments. Government Schemes pertaining to a topic, allocation as per the budget, developments/achievements till now, way forward, etc
4. LANGAUAGE AND GRAMMAR: Errors of subject verb agreement, preposition, Spellings, appropriate spacing, capitalization and punctuations must be checked.

GRADING CRITERIA:
Please grade the essay on the below parameters and provide a Overall Grades as A+ or A or B+ or B or C or D.
1. LANGAUAGE AND GRAMMAR: Please check subject verb agreement errors.Also Spellings, appropriate spacing, capitalization and punctuations must be checked.
2. WORD LIMIT: The total word count is 300 words. Please  deduct -0.5 marks each for every increase or decrease of 10 words.
3.CONTENT : Please analyse the matter as per the current developments. Government Schemes pertaining to a topic, allocation as per the budget, developments/achievements till now, way forward, etc, whether they are mentioned or not.If the content lacks any of the points mentioned deduct -0.5 marks each.
4. TITLE : Please deduct 3 marks if either title or serial number of the chosen essay topic is not  mentioned.


ESSAY CONTENT:
```
{context}

```

Please provide the feedback in the format specified below :
FEEDBACK:

1.Prescribed word count: round 300 words

2.Student word count:  specify the number of words present in the essay content specified within 3 backticks

3.Try to stay within +/- 10 words of the prescribed word limit : If the student word count is within a + or - 10 words then mention "Well Done" otherwise mention "Word Limit not adhered"

4.Title: If tile is present then mention " Title provided" otherwise mention "No Title provided"

5. Content and Structure: Provide points between -5 to 10 based on the GRADING CRITERIA for CONTENT

6. Language: Provide the points based on the GRADING CRITERIA for LANGAUAGE AND GRAMMAR.Also provide spelling errors in RED and provide the correct answer next to the incorrect words in GREEN BOLD.
{format_instructions}
"""

In [11]:
llm = ChatOpenAI(temperature=0.0)

prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template(template)
    ],
    input_variables=["context"],
    partial_variables={"format_instructions": format_instructions},
    output_parser=output_parser # here we add the output parser to the Prompt template
)

In [12]:
context = """(c) Financial Inclusion and development of the poor

 India is one of the countries with highly inequitable distribution of wealth.
 A report pointed out, that only 1% of the wealthy Indians hold
 90% of the total income of our country. India, due to its population
 explosion has one of the lowest per capita income.
 The poor lacks even the basic services necessary for a healthy life.
 We ranked 132 in the HDI index, which signifies, that a lot of
 groundwork needs to be done to improve even the basic Human
 Development factors.

 Financial Inclusion refers to providing basic financial services such
 as banking, insurance, pension etc. to the poorest and the
 vulnerable strata of the society. Most of the rural poor lack the
 access to basic financial services. The Union Government came
 out with PM Jan Dhan Yojana in 2014 to provide banking services
 to the poor. It provided A basic savings bank deposit account with
 no minimum deposit requirements. It was the first step in bringing
 a host of financial services thereafter.

 In 2015, THE government came out with insurance sector schemes such as PM Suraksha Bima Yojana
 and PM Jeevan Jyoti Beema Yojana. The former was a Personal accident
 insurance having coverage upto Rs 2 lacs for death at a premium
 of Rs 20/-and the latter being a life insurance scheme at a premium
 of Rs 436/- for Sum Insured of Rs 2 lacs. This followed with BY schemes
 in the pension sector viz. Atal Pension Yojana, PM SVanidhi, PM Shram
 Yogi Maan dhan and recent amendments in NPS.

 All the above government initiatives point to the efforts made for
 the financial inclusion of the poor. These initiatives have not only helped
 poor gain access to basic financial services, but also for targeted
 delivery of DBT and subsidy services directly in the bank accounts
 of the beneficiaries
"""

In [16]:
print(prompt.format(context=context ,
                    history=None))

Human: 
Evaluate the context of the essay specified in 3 bacticks on the following criteria:
ESSAY CRITERIA:
1. TITLE : The essay must have a title.
2. WORD LIMIT: The word count in the essay must be between 290 to 310 words
3. CONTENT : The content must have the matter as per the current developments. Government Schemes pertaining to a topic, allocation as per the budget, developments/achievements till now, way forward, etc
4. LANGAUAGE AND GRAMMAR: Errors of subject verb agreement, preposition, Spellings, appropriate spacing, capitalization and punctuations must be checked. 

GRADING CRITERIA:
Please grade the essay on the below parameters and provide a Overall Grades as A+ or A or B+ or B or C or D.
1. LANGAUAGE AND GRAMMAR: Please check subject verb agreement errors.Also Spellings, appropriate spacing, capitalization and punctuations must be checked.
2. WORD LIMIT: The total word count is 300 words. Please  deduct -0.5 marks each for every increase or decrease of 10 words.
3.CONTEN

In [17]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm,
                 prompt=prompt)

In [18]:
response = chain.predict_and_parse(context=context,history=None)

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


In [19]:
print(response)

{'Prescribed Word Count': '300', 'Student Word Count': '298', 'Try to stay within +/- 10 words of the prescribed word limit': 'Well Done', 'Title': 'Title provided', 'Content and Structure': '8', 'Language': "8 (Errors: 'THE' should be 'the', 'strata' should be 'stratum', 'viz.' should be 'viz', 'Maan dhan' should be 'Maandhan', 'DBT' should be 'Direct Benefit Transfer', 'beneficiaries' should be 'beneficiary's')"}


In [20]:
response

{'Prescribed Word Count': '300',
 'Student Word Count': '298',
 'Try to stay within +/- 10 words of the prescribed word limit': 'Well Done',
 'Title': 'Title provided',
 'Content and Structure': '8',
 'Language': "8 (Errors: 'THE' should be 'the', 'strata' should be 'stratum', 'viz.' should be 'viz', 'Maan dhan' should be 'Maandhan', 'DBT' should be 'Direct Benefit Transfer', 'beneficiaries' should be 'beneficiary's')"}

In [21]:
type(response)

dict

## PRECIS

In [81]:
p_word_count_schema = ResponseSchema(name="Prescribed Word Count",
                                description = "This is Word count that has been prescribed as 300 Words ")
#
s_word_count_schema = ResponseSchema(name="Student Word Count",
                                  description = "This is the word count thats specifies the number of words present in the essay content specified within 3 backticks")
#
s_word_limit_schema = ResponseSchema(name="Try to stay within +/- 10 words of the prescribed word limit",
                                     description="This is the word limit that the student has adhered the word limit or exceeded the word limit")
#
title_schema = ResponseSchema(name="Title",
                              description="Title of the Essay in case provided")
#
content_schema = ResponseSchema(name="Content and Structure",
                                description="This provides points between -5 to 10 based on the GRADING CRITERIA for CONTENT")
#
language_schema = ResponseSchema(name="Language",
                                 description="This provides the points based on the GRADING CRITERIA for LANGAUAGE AND GRAMMAR.Also provide spelling errors in RED and provide the correct answer next to the incorrect words in GREEN BOLD.")

response_schemas = [p_word_count_schema,
                    s_word_count_schema,
                    s_word_limit_schema,
                    title_schema,
                    content_schema,
                    language_schema]

In [82]:
template = """Evaluate the content of the Precis provided below enclosed in 3 backticks.

EVALUATION CRITERIA FOR WRITING PRECIS:

1. TITLE : Title must be present. The title should be related to the main idea of the passage. If no title provided deduct 2 marks.
2. LANGUAGE : The content must be in past tense reported speech.
3. CONTENT : Data and numbers should not be copied but interpreted. For Example :90% of the population – Majority of the population,0.3% profits – Marginal profits
4. The precis Must be written in 1 paragraph
5. The precis content should follow the chronology of the passage
6. The contentof the precis must be written in own words.
7. The precis must be reduced to 1/3rd length of the original passage
8. The precis must include all relevant information of the passage

PASSAGE
```
{input}

```
Please provide the feedback in the format specified below :
FEEDBACK:
1.Prescribed word count: 173 words
2.Student word count:  specify the number of words present in the PRECIS content specified within 3 backticks
3 Try to stay within +/- 10 words of the prescribed word limit : If the student word count is within a + or - 10 words then mention "Well Done" otherwise mention "Word Limit not adhered".
4.Title: If tile is present then mention " Title provided" otherwise mention "No Title provided".Deduct 2 marks(-2)
5. LANGUAGE ERRORS: If the content of precis provided in 3 backticks is not specified in past tense reported speech provide feedback as "Write in past tense reported speech, always." Deduct 5 marks (-5)
6. CONTENT AND STRUCTURE: if the content of precis provided in 3 backticks does not follow the points 4,5,6,7,8 provided in the EVALUATION CRITERIA FOR WIRTING PRECIS then provide the feedback as "Content ans Structure format is not adhered".Al;so check if positive and negative sentiments have been mentioned. If not found then specify the sentiments which is missing and deduct 3 marks(-3)

{format_instructions}
"""

In [83]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

In [84]:
llm = ChatOpenAI(temperature=0.0)

prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template(template)
    ],
    input_variables=["input"],
    partial_variables={"format_instructions": format_instructions},
    output_parser=output_parser # here we add the output parser to the Prompt template
)

In [45]:
context =  """(c)Social Media and the sense of false self:

 On Social Media (no capitalization) we always try to show ourselves a PROJECT ourselves as person of significance
 and permeance, always trying to beautify our bodies, try to amaze huge wealth
 in order to buy luxurious items and amenities. All of this we do to make ourselves
 special and MORE significant than others. But when we can not maintain the image we
 have created in front of others it leads to negative feelings for ourselves and
 can also break our self esteem to a larger extend. All this feelings to remain special
 comes from the belief that we are separate from others but it is not true
 as we don't matter in this big unfathomable universe (clumsy sentence, put full stop) the only solution to let
 go of our entitlement is to accept who we are and what we try to become
 and we should accept others also in the same manner. We should treat others
 with kindness and empathy and not try to make others miserable because of the  fake pompous self we have created for ourselves.
"""

In [46]:
print(prompt.format(input=context ,
                    history=None))

Human: Evaluate the content of the Precis provided below enclosed in 3 backticks.

EVALUATION CRITERIA FOR WRITING PRECIS:

1. TITLE : Title must be present. The title should be related to the main idea of the passage. If no title provided deduct 2 marks.
2. LANGUAGE : The content must be in past tense reported speech.
3. CONTENT : Data and numbers should not be copied but interpreted. For Example :90% of the population – Majority of the population,0.3% profits – Marginal profits
4. The precis Must be written in 1 paragraph
5. The precis content should follow the chronology of the passage
6. The contentof the precis must be written in own words.
7. The precis must be reduced to 1/3rd length of the original passage
8. The precis must include all relevant information of the passage

PASSAGE
```
(c)Social Media and the sense of false self:

 On Social Media (no capitalization) we always try to show ourselves a PROJECT ourselves as person of significance
 and permeance, always trying to be

In [47]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm,prompt=prompt)
#
response = chain.predict_and_parse(input=context,history=None)
response

{'Prescribed Word Count': '173 words',
 'Student Word Count': '87 words',
 'Try to stay within +/- 10 words of the prescribed word limit': 'Well Done',
 'Title': 'No Title provided (-2)',
 'Content and Structure': 'Content and Structure format is not adhered. Positive and negative sentiments are missing (-3)',
 'Language': 'Write in past tense reported speech, always (-5)'}

## READING COMPREHENSION

In [74]:
p_word_count_schema = ResponseSchema(name="Prescribed Word Count",
                                description = "This is Word count that has been prescribed as 300 Words ")
#
s_word_count_schema = ResponseSchema(name="Student Word Count",
                                  description = "This is the word count thats specifies the number of words present in the essay content specified within 3 backticks")
#
answer_schema = ResponseSchema(name="Answer Format",
                                     description="This provides the answer provided by the student. Good format of answer should be, background of the situation/issue mentioned in the question to be given, followed by the required statistics or information specific to the question.")

response_schemas = [p_word_count_schema,
                    s_word_count_schema,
                    answer_schema]

In [66]:
template ="""Evaluate the comprehending as well as writing skills of the student based on the content enclosed in 3 backticks based on the passage provided enclosed within 3 single quotes '''

EVALUATION CRITERIA :
1. The content written should in your own words.
2. Change language as much as possible
3. DO NOT COPY PASTE from the passage.
4. Please  Elaborate your underrstanding of the passage enclosed within 3 single quotes ''' . Word limit should be between  70-100 words.
5. The word  limit should not exceed 100 words.
6. The Answer provided by the student enclosed in 3 backticksshould have the background of the situation/issue mentioned in the question to be given, followed by the required statistics or information specific to the question.
7. Please do not provide any other information from outside. The answer should have the contect strictly from the passage provided enclosed in 3 single quotes '''.
8. Please specify the number of words present in the PRECIS content specified within 3 backticks ```

PASSAGE:
'''
{input}
'''
ANSWER:
```
{answer}

```
Please provide the feedback in the following format:
FEEDBACK:
1. Prescribed word count: Between 70 -100 words
2. Student word count:  specify the number of words present in the ANSWER content specified within 3 backticks
3. Answer Format:  Good format of answer should be, background of the situation/issue mentioned in the question to be given, followed by the required statistics or information specific to the question.

{format_instructions}
"""

In [75]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

In [76]:
llm = ChatOpenAI(temperature=0.0)

prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template(template)
    ],
    input_variables=["input","answer"],
    partial_variables={"format_instructions": format_instructions},
    output_parser=output_parser # here we add the output parser to the Prompt template
)

In [77]:
input = """The government and UN agencies, in order to tackle the increasing malnutrition, needs NEED to work on strengthen STRENGTHENING the nutritional safety nets of the poor. Government should increase the budgets for the ICDS and MDM schemes in accordance with the rise in inflation, and continue giving subsidised rations to THE poor under PMGKAY. The Finance Ministry and NITI Aayog need to work on arranging funds for the same. At the same time, the government needs to find ways to reduce wastage in food, make agriculture logistics more efficient and also try to reduce food prices by reducing the margins of the traders. The government should also work on subsidised distribution of essential micronutrients to the poor.
"""

In [78]:
answer = """Low income hence poor health of children, The governments and the UN agencies should help by increasing the budget of government (ICDS) and mid­day meal (MDM) schemes in congruence with price rise, budget and duration of PMGKAY."""

In [79]:
print(prompt.format(input=input,answer=answer,
                    history=None))

Human: Evaluate the comprehending as well as writing skills of the student based on the content enclosed in 3 backticks based on the passage provided enclosed within 3 single quotes '''

EVALUATION CRITERIA : 
1. The content written should in your own words. 
2. Change language as much as possible 
3. DO NOT COPY PASTE from the passage.
4. Please  Elaborate your underrstanding of the passage enclosed within 3 single quotes ''' . Word limit should be between  70-100 words.
5. The word  limit should not exceed 100 words.
6. The Answer provided by the student enclosed in 3 backticksshould have the background of the situation/issue mentioned in the question to be given, followed by the required statistics or information specific to the question.
7. Please do not provide any other information from outside. The answer should have the contect strictly from the passage provided enclosed in 3 single quotes '''.
8. Please specify the number of words present in the PRECIS content specified within

In [80]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm,prompt=prompt)
#
response = chain.predict_and_parse(input=input,answer=answer,history=None)
response

{'Prescribed Word Count': 'Between 70 -100 words',
 'Student Word Count': '20',
 'Answer Format': 'The answer provided by the student is too brief and lacks the required elaboration and explanation. It does not provide any statistics or information specific to the question. The student needs to work on improving their comprehension and writing skills.'}